In [ ]:
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import IPython
from scipy.io import wavfile
%matplotlib inline

In [ ]:
def load_data(filename):
    sr, s = wavfile.read(filename)
    s = s/np.max(s)
    return s, sr

In [ ]:
def add_noise(x,SNR):
    Ps=np.sum(x**2)
    noise_variance = Ps*10**(-SNR/10)
    noise = np.sqrt(noise_variance)*np.random.randn(len(x))
    return x+noise

In [ ]:
def frame_split(x,frame_size):
    # Overlap : the choice of analysis and synthesis windows is as in the STFT course
    # Careful that it is only consistent for a 50% overlap
    overlap_ratio = 0.5
    overlap = int(overlap_ratio*frame_size)
    w_a = sig.windows.cosine(frame_size)
    w_s = sig.windows.cosine(frame_size)

    # Compute the number of frames and pad the input signal at the end with zeros
    # to work only with full frames
    nframes = int(np.ceil((len(x)-overlap)/(frame_size-overlap)))
    
    padded_size = int((frame_size-overlap)*nframes+frame_size)
    x_padded = np.concatenate((x,np.zeros(padded_size-len(x))))
   
    
    list_frames = [[frame*(frame_size-overlap),frame*(frame_size-overlap)+frame_size] for frame in range(nframes)]
    
    return list_frames, x_padded, w_a, w_s

In [ ]:
from utils import autocorrelation
def lpc_analysis(s,p):
    """Compute the parameters a_k of the p-order all-pole filter by solving the linear system obtained by MAP with
    observed signal s. Following case 2 from the article"""
    from scipy.linalg import solve, toeplitz
    N = len(s)

    # Efficient building of the linear system by using the fact that A is a symmetric toeplitz, and its diagonals, as well
    # as the RHS, just contain the autocorrelation of s for various intervals
    # 1. Compute autocorrelations
    R = np.zeros(p+1)
    for k in range(len(R)):
        R[k] = autocorrelation(s,k)
    # 2. Build LHS, RHS and solve
    A = toeplitz(R[0:p])
    b = R[1:p+1]
    a = solve(A,b)

    # 3. Compute the gain factor for the excitation. Pad s with p zeros at the beginning (assumption on the S_I)
    padded_s = np.concatenate((np.zeros(p),s))
    g2 = 0
    for n in range(0,N):
        g2 = g2 + (padded_s[n+p] - a@np.flip(padded_s[n:n+p],0))**2
    g = np.sqrt(1/N*g2)
    return a,g

In [ ]:
def denoise_frame(x,p,sr,iterations):
    """Denoise a frame in the STFT domain by applying iterative Wiener filtering
    and approximating the PSD of the signal through the use of the all-pole model
    """
    P_n = 9e-4 
    #For now : estimator of PSD of the noise. Should be implemented with more care to improve results
    #For gaussian noise : P_n = sigma_n^2
    
    dft = np.fft.rfft(x) # 
    omega = np.linspace(0,np.pi,len(dft))
    
    # First signal iterant is the original (noisy) signal
    x_i = x
    for it in range(iterations):
        a,g = lpc_analysis(x_i,p) # Compute the coefficients of the all-pole filter from the current iterant

        # Compute the estimation of the PSD of the speech signal
        def PSD(omega):
            k = np.arange(p)
            return g**2/(np.abs(1-a@np.exp(-1j*k*omega)))**2
        vec_PSD = np.vectorize(PSD) 
        
        # Compute the transfer function of the Wiener Filter
        transfer = vec_PSD(omega)/(vec_PSD(omega)+P_n)
        
        # Filter the original signal with the Wiener filter, and compute a
        # next iterant
        filtered_dft = transfer*dft
        x_i = np.fft.irfft(filtered_dft)
    return x_i

In [ ]:
# Parameters 
p = 10 #Order of the all pole filter
frame_size = 400 # Window size
iterations = 5 # Number of iterations of the Wiener Filter

# Preparing the sample
y_clean, sr = load_data('../samples/arctic_b0443.wav')
# For now : choose the SNR (Db) and add corresponding noise to the signal
SNR = 60
y = add_noise(y_clean,SNR)

# Get the list of frames for the STFT filtering
list_frames,y_padded,w_a,w_s = frame_split(y,frame_size)
n_frames = len(list_frames)

s = np.zeros(y_padded.shape)
for frame in range(n_frames):
    start,end = list_frames[frame]
    s[start:end] = w_s*denoise_frame(w_a*y_padded[start:end],p,sr,iterations)


In [ ]:
# Plot and play the noisy signal
plt.plot(y)
IPython.display.Audio(y,rate=sr)

In [ ]:
#Plot and play the denoised signal
plt.plot(s)
IPython.display.Audio(s,rate=sr)